In [7]:
import pandas as pd 
xlsx = pd.read_csv('vrp_ekz_prog.csv')

lgt_csv = pd.read_csv('lgt_AO(2019-2025).csv')
lgt = lgt_csv['0'] 

i=0
c=100

newdf = pd.DataFrame(columns=['c1'], index=range(21))
while i<=20:
    c = (c*((lgt[i]-100) * 1.4288808951077208+100))/100
    newdf.loc[i].c1 = c
    i=i+1 
ltrnS=newdf['c1']
l = newdf['c1']
trn_l = l.shift(1)

trn_l_b =trn_l.loc[xlsx.o_script.str.match(r'базовый')].reset_index(drop = True)
trn_l_o =trn_l.loc[xlsx.o_script.str.match(r'оптимистичный')].reset_index(drop = True)
trn_l_p =trn_l.loc[xlsx.o_script.str.match(r'пессимистичный')].reset_index(drop = True)
trn_l_b[0]= 100
trn_l_o[0]= 100
trn_l_p[0]= 100
trn_l = trn_l_b.append(trn_l_o).append(trn_l_p).reset_index(drop = True)

limt = ltrnS/trn_l*100

limt.to_csv(r'limt_AO(2019-2025).csv', index=False)